# 海龜交易策略回測範例

這個筆記本示範如何使用我們的交易策略回測框架來測試海龜交易策略。

In [ ]:
# 導入必要的套件
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import fetch_data, plot_price_and_signals, print_strategy_summary
from strategies.turtle_strategy import TurtleStrategy
from backtest_engine import BacktestEngine
from config.config import get_config

## 1. 獲取市場資料

In [ ]:
# 選擇要測試的股票
symbol = 'QQQ'
period = '5y'  # 5年的資料

# 獲取資料
print(f"正在獲取 {symbol} 的資料...")
data = fetch_data(symbol, period=period)
print(f"獲取了 {len(data)} 筆資料")
data.head()

## 2. 創建交易策略

In [ ]:
# 創建海龜交易策略
strategy = TurtleStrategy(
    name="海龜策略 (20/10)",
    entry_window=20,
    exit_window=10,
    initial_capital=100000
)

print(f"策略名稱: {strategy.name}")
print(f"進場窗口: {strategy.entry_window} 天")
print(f"出場窗口: {strategy.exit_window} 天")
print(f"初始資金: ${strategy.initial_capital:,.2f}")

## 3. 運行回測

In [ ]:
# 創建回測引擎
backtest_config = get_config('backtest')
engine = BacktestEngine(
    initial_capital=backtest_config['initial_capital'],
    commission=backtest_config['commission']
)

# 運行回測
print("開始回測...")
result = engine.run_backtest(strategy, data, symbol)
print("回測完成!")

## 4. 分析結果

In [ ]:
# 顯示策略摘要
performance = result['performance']
print_strategy_summary(strategy.name, performance)

In [ ]:
# 顯示交易記錄
trades_df = pd.DataFrame(result['trades'])
if not trades_df.empty:
    print(f"
總交易次數: {len(trades_df)}")
    print("
前10筆交易:")
    print(trades_df.head(10))
else:
    print("
沒有執行任何交易")

## 5. 視覺化分析

In [ ]:
# 繪製價格和交易信號
signals_data = result['signals_data']
plot_price_and_signals(signals_data, f"{strategy.name} - {symbol}")

In [ ]:
# 繪製投資組合價值變化
portfolio_value = result['portfolio_value']
if portfolio_value:
    plt.figure(figsize=(12, 6))
    
    # 投資組合價值
    portfolio_series = pd.Series(portfolio_value, index=data.index[:len(portfolio_value)])
    plt.plot(portfolio_series.index, portfolio_series.values, label='投資組合價值', linewidth=2)
    
    # 買入持有策略比較
    buy_hold_value = (data['Close'] / data['Close'].iloc[0]) * strategy.initial_capital
    plt.plot(data.index, buy_hold_value, label='買入持有', linewidth=2, alpha=0.7)
    
    plt.title(f'{strategy.name} - 投資組合價值 vs 買入持有')
    plt.xlabel('日期')
    plt.ylabel('價值 ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 計算買入持有的績效
    buy_hold_return = (buy_hold_value.iloc[-1] / strategy.initial_capital) - 1
    strategy_return = performance.get('total_return', 0)
    
    print(f"
績效比較:")
    print(f"策略總回報: {strategy_return:.2%}")
    print(f"買入持有回報: {buy_hold_return:.2%}")
    print(f"超額回報: {(strategy_return - buy_hold_return):.2%}")